<a href="https://colab.research.google.com/github/YaqubH/Football_Matches_Predictor/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests

In [11]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [12]:
data = requests.get(standings_url)

In [13]:
from bs4 import BeautifulSoup

In [14]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [15]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [16]:
data = requests.get(team_urls[0])

In [17]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [18]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [19]:
data = requests.get(f"https://fbref.com{links[0]}")

In [20]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()